In [1]:
from coffea import processor, util

%matplotlib inline
import matplotlib.pyplot as plt

from boostedhiggs import HbbProcessor

In [2]:
files = {
    'data': [
        'data/EGamma-Run2018D.root'
    ],
}

p = HbbProcessor(year='2018')
exe_config = {
    'flatten': True,
    'workers': 4,
    'savemetrics': True,
    'compression': 0,
    'nano': True,
}

output, metrics = processor.run_uproot_job(files, 'Events', p, processor.futures_executor, exe_config)

/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users

In [3]:
evtrate = metrics['entries'].value / metrics['processtime'].value
print("Events / s / thread: %.0f" % evtrate)

Events / s / thread: 94097


In [4]:
files = {
    'ttbar': [
        'data/TTToHadronic_TuneCP5_13TeV-powheg-pythia8_10X.root'
    ],
    'zjets':[
        'data/ZJetsToQQ_HT600to800.root'
    ]
}

p = HbbProcessor(year='2017')
exe_config = {
    'flatten': True,
    'workers': 4,
    'savemetrics': True,
    'compression': 0,
    'nano': True,
}

output_mc, metrics = processor.run_uproot_job(files, 'Events', p, processor.futures_executor, exe_config)
output += output_mc

/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ncsmith/src/awkward-array/awkward/array/jagged.py:1035: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/

In [5]:
evtrate = metrics['entries'].value / metrics['processtime'].value
print("Events / s / thread: %.0f" % evtrate)

Events / s / thread: 22151


In [6]:
output

{'cutflow': defaultdict_accumulator(float,
                         {'none': 3672366.0,
                          'jetkin': 19178.0,
                          'trigger': 5696.0,
                          'jetid': 5694.0,
                          'n2ddt': 1756.0,
                          'antiak4btagMediumOppHem': 1720.0,
                          'met': 1598.0,
                          'noleptons': 1214.0})}